In [2]:
from IPython.display import display
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

data_path = 'C://3rd-ml100marathon-midterm/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

train_Y = df_train['poi']
ids = df_test['name']
df_train = df_train.drop(['name', 'poi'], axis=1)
df_test = df_test.drop(['name'], axis=1)
df = pd.concat([df_train, df_test])
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [3]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(20))
na_check(df)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_messages,41.095890
from_poi_to_this_person,41.095890
from_this_person_to_poi,41.095890


In [4]:
df['bonus_group'] = df['bonus'].map(lambda s: 1 if s > df.bonus.mean() else 0)
df['high_salary_group'] = df['salary'].map(lambda s: 1 if s > df.salary.mean() else 0)
df['no_email_data'] = df['email_address'].isna()
df['no_email_data'] = df['no_email_data'].map(lambda s: 1 if s else 0)
df = df.drop(['email_address'] , axis=1)
df = df.fillna(0)

In [5]:
df = MinMaxScaler().fit_transform(df)
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.25, random_state=42)

In [6]:
# 設定要訓練的超參數組合
tol = np.arange(0.0001,1,0.0001)
penalty = ['l1', 'l2']
fit_intercept = [True, False]
C = np.arange(0.01,1,0.01)
param_dist = dict(tol=tol, penalty=penalty, fit_intercept=fit_intercept, C=C)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
random_search = RandomizedSearchCV(LogisticRegression(), param_dist,  n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
acc = 0
while acc < 0.8:
    search_result = random_search.fit(x_train, y_train)
    lr = LogisticRegression(
        tol=search_result.best_params_['tol'], 
        penalty=search_result.best_params_['penalty'], 
        fit_intercept=search_result.best_params_['fit_intercept'], 
        C=search_result.best_params_['C'])
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(acc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


0.8275862068965517


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    3.6s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    3.6s finished


In [7]:
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': lr_pred})
sub['poi'] 
sub.to_csv('Midterm.csv', index=False) 